## Найдем сетку по домам с центрами масс

У нас много домов и некоторые расчеты могут происходить долго, поэтому построим сетку с разным шагом (100 метров, 500 метров, 1 км, 2 км) на которую мы наложим дома с населением, посчитав в каждой ячейке центр масс. Делаем это, чтобы уменьшить объем данных для ускорения работы алгоритмов

In [1]:
import os
import json
import pandas as pd
import numpy as np
import sys

sys.path.append('/Users/marina/Documents/my_projects/hackathon_postomat')
from postomat_optimisation.src.postamats.utils.connections import DB
from postomat_optimisation.src.postamats.utils.helpers import haversine

pd.set_option('display.max_columns', None)

In [2]:
CONFIG_PATH = "//Users/marina/Documents/my_projects/hackathon_postomat/db_config.json"
with open(CONFIG_PATH) as f:
    db_config = json.load(f)

db = DB(db_config)

In [38]:
df = db.get_table_from_bd('apartment_houses_all_data')

Connection to PostgreSQL DB successful
/Users/marina/Documents/my_projects/hackathon_postomat/postomat_optimisation/src/postamats/utils/connections.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table_name}', connection)


In [40]:
df['population'] = df['total_area']/22

In [41]:
df.head(2)

,object_type,on_moscow_territory,address,simple_address,street,local_object_type,local_object_num,korpus_num,stroenie_num,adm_area,district,num_addr_register,date_addr_register,guid_fias,date_fias,kad_n,kad_zu,kladr_code,addr_status,geodata,lat,lon,object_id,address_gis,address_gis_code,oktmo_code,management_method,management_ogrn,management_kpp,management_name,house_type,condition,total_area,living_area,demolition_date,cadastral_num,guid_house,population
0,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","улица Бахрушина, дом 10, строение 3",улица Бахрушина,дом,10,None,3,Центральный административный округ,муниципальный округ Замоскворечье,1001314,24.05.2002,FFBC94D2-7371-426A-9AAA-2F606FD1BE86,24.11.2011,77:01:0002012:1017,None,77000000000081800,Внесён в ГКН,"{{37.6347869776043,55.735330239525},{37.634592...",55.735592,37.634462,a1b50bfc376ea37edebd16ddd131b106bf8d8c846983a9...,"115054, Москва г, ул. Бахрушина, д. 10, строен...",ffbc94d2-7371-426a-9aaa-2f606fd1be86,45376000,УО,5137746116646,770501001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,2184.3,2099.5,None,None,22e4df6f-c568-4b7d-b119-cdb90deddf1c,99.286364
1,многоквартирный дом,да,"Российская Федерация, город Москва, внутригоро...","улица Артамонова, дом 8, корпус 2",улица Артамонова,дом,8,2,None,Западный административный округ,муниципальный округ Фили-Давыдково,7103698,24.12.2003,0DA29A7F-E80F-415A-8C73-4C780014EA98,27.02.2012,77:07:0009003:1020,None,77000000000076700,Внесён в ГКН,"{{37.4572289466062,55.720043356036},{37.456852...",55.720403,37.457119,9a11acf2996873fc758e8f2454f47250bd5239153c8dc8...,"121357, Москва г, ул. Артамонова, д. 8, корп. 2",0da29a7f-e80f-415a-8c73-4c780014ea98,45329000,УО,1157746682610,773101001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,3187.0,NaN,None,None,769ca40a-b4a8-4991-9a51-42117cfddd44,144.863636


In [43]:
def find_degreee_to_distance(df):
    "Функция, которая возвращает чему равен 1 градус по широте и долготе в градусах"
    lat_min = df.lat.min()
    lat_max = df.lat.max()
    lon_min = df.lon.min()
    lon_max = df.lon.max()
    lat_length= haversine(lat_min, lon_min, lat_max, lon_min)
    lon_length= haversine(lat_min, lon_min, lat_min, lon_max)
    lat_km = lat_length/1000/(lat_max-lat_min)
    lon_km = lon_length/1000/(lon_max-lon_min)
    print(f'latitude 1 degree = {lat_km} km', f'longitude 1 degree = {lon_km} km') 

    return lat_km, lon_km

In [45]:
lat_km, lon_km = find_degreee_to_distance(df)
DISTANCE_TO_DEGREE = {'lat': 1/lat_km, 'lon': 1/lon_km}

latitude 1 degree = 111.06521377455095 km longitude 1 degree = 62.91680428045886 km


In [46]:
def make_net_with_center_mass(df_homes, step, distance_to_degree):
    """
    Функция, которая накладывает объекты (дома) на сетку и в каждой ячейке считает центр масс
    В df_homes обязаны быть поля population, lat, lon

    """
    df = df_homes.copy()
    df.columns = [column.lower() for column in df.columns]
    
    step_lon = step * distance_to_degree['lon']
    step_lat = step * distance_to_degree['lat']

    df['lat_n'] = df.lat // step_lat
    df['lon_n'] = df.lon // step_lon
    df['lat_n'] = df['lat_n'].astype('int')
    df['lon_n'] = df['lon_n'].astype('int')
    df['lat_n_lon_n'] = df['lat_n'].astype('str') + '_' + df['lon_n'].astype('str')
    df['step'] = step

    df['id_center_mass'] = df['lat_n_lon_n'] + '_' + df['step'].astype(str)

    df['lat_population'] = df['lat']*df['population']
    df['lon_population'] = df['lon']*df['population']
    df_agg = df.groupby(['id_center_mass']).agg({'population':'sum','lat_population':'sum','lon_population':'sum'}).reset_index().rename({'population':'sum_population'}, axis=1)

    df_agg['lat'] = df_agg['lat_population']/df_agg['sum_population']
    df_agg['lon'] = df_agg['lon_population']/df_agg['sum_population']
    df_agg['population'] = df_agg['sum_population']
    df_agg = df_agg[['id_center_mass','lat','lon','population']]
    df_agg['step'] = step

    return df_agg

In [60]:
step = 0.1
df_result_01 = make_net_with_center_mass(df, step)

In [61]:
step = 0.5
df_result_05 = make_net_with_center_mass(df, step)

In [62]:
step = 1
df_result_1 = make_net_with_center_mass(df, step)

In [63]:
step = 2
df_result_2 = make_net_with_center_mass(df, step)

In [64]:
df_result = pd.concat([df_result_01, df_result_05, df_result_1, df_result_2])

In [53]:
df_result.head()

,id_center_mass,lat,lon,population,step
0,61634_23649_0.1,55.494308,37.588680,27.136364,0.1
1,61635_23649_0.1,55.495040,37.587754,3.436364,0.1
2,61635_23650_0.1,55.494963,37.589354,7.436364,0.1
3,61635_23651_0.1,55.494637,37.591560,2.818182,0.1
4,61636_23649_0.1,55.496060,37.589255,2.104545,0.1


In [66]:
db.load_to_bd(df_result, 'centers_mass')

Ниже кусок кода для итогового общего ETL скрипта

In [10]:
def find_degreee_to_distance(df):
    "Функция, которая возвращает чему равен 1 градус по широте и долготе в градусах"
    lat_min = df.lat.min()
    lat_max = df.lat.max()
    lon_min = df.lon.min()
    lon_max = df.lon.max()
    lat_length= haversine(lat_min, lon_min, lat_max, lon_min)
    lon_length= haversine(lat_min, lon_min, lat_min, lon_max)
    lat_km = lat_length/1000/(lat_max-lat_min)
    lon_km = lon_length/1000/(lon_max-lon_min)
    print(f'latitude 1 degree = {lat_km} km', f'longitude 1 degree = {lon_km} km') 

    return lat_km, lon_km

def make_net_with_center_mass(df_homes, step, distance_to_degree):
    """
    Функция, которая накладывает объекты (дома) на имеющуюся сетку и в каждой ячейке считает центр масс
    В df_homes обязаны быть поля population, lat, lon

    """
    df = df_homes.copy()
    df.columns = [column.lower() for column in df.columns]
    
    step_lon = step * distance_to_degree['lon']
    step_lat = step * distance_to_degree['lat']

    df['lat_n'] = df.lat // step_lat
    df['lon_n'] = df.lon // step_lon
    df['lat_n'] = df['lat_n'].astype('int')
    df['lon_n'] = df['lon_n'].astype('int')
    df['lat_n_lon_n'] = df['lat_n'].astype('str') + '_' + df['lon_n'].astype('str')
    df['step'] = step

    df['id_center_mass'] = df['lat_n_lon_n'] + '_' + df['step'].astype(str)

    df['lat_population'] = df['lat']*df['population']
    df['lon_population'] = df['lon']*df['population']
    df_agg = df.groupby(['id_center_mass']).agg({'population':'sum','lat_population':'sum','lon_population':'sum'}).reset_index().rename({'population':'sum_population'}, axis=1)

    df_agg['lat'] = df_agg['lat_population']/df_agg['sum_population']
    df_agg['lon'] = df_agg['lon_population']/df_agg['sum_population']
    df_agg['population'] = df_agg['sum_population']
    df_agg = df_agg[['id_center_mass','lat','lon','population']]
    df_agg['step'] = step

    return df_agg


LIST_STEP = [0.1, 0.5, 1, 2] # список размеров величины шага в км в сетке, которую мы накладываем на дома

def make_final_table_with_center_mass(db):
    df = db.get_table_from_bd('apartment_houses_all_data')
    df['population'] = df['total_area']/22 #temporary

    lat_km, lon_km = find_degreee_to_distance(df)
    DISTANCE_TO_DEGREE = {'lat': 1/lat_km, 'lon': 1/lon_km}
    df_result = pd.DataFrame()
    for step in LIST_STEP:
        df_result_step = make_net_with_center_mass(df, step, DISTANCE_TO_DEGREE)
        df_result = pd.concat([df_result_step,df_result])
    db.load_to_bd(df_result, 'centers_mass')


In [11]:
# итоговый вызов функции, чтобы получить сетку с центрами масс
make_final_table_with_center_mass(db)

Connection to PostgreSQL DB successful
/Users/marina/Documents/my_projects/hackathon_postomat/postomat_optimisation/src/postamats/utils/connections.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table_name}', connection)
latitude 1 degree = 111.06521377455095 km longitude 1 degree = 62.91680428045886 km
